In [2]:
!pip install faiss-cpu sentence-transformers transformers

In [10]:
!git clone https://github.com/Rithvik50/Legal_AEye-Opener.git

fatal: destination path 'Legal_AEye-Opener' already exists and is not an empty directory.


In [13]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import json
from transformers import pipeline

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Load FAISS index and metadata
def load_faiss():
    global index, documents
    index = faiss.read_index("/content/Legal_AEye-Opener/law_faiss.index")
    with open("/content/Legal_AEye-Opener/law_metadata.json", "r", encoding="utf-8") as f:
        documents = json.load(f)

# Function to retrieve relevant sections
def retrieve_sections(query, law_type=None, top_k=3):
    query_embedding = model.encode(query).astype(np.float32)
    distances, indices = index.search(np.array([query_embedding]), top_k)

    results = []
    for i in range(top_k):
        if indices[0][i] != -1:
            section = documents[indices[0][i]]
            if law_type is None or section["law_type"].lower() == law_type.lower():
                results.append({
                    "law_type": section["law_type"],
                    "chapter_number": section["chapter_number"],
                    "chapter_title": section["chapter_title"],
                    "content": section["content"],
                    "source_url": section["source_url"]
                })
    return results

# Summarization function using Hugging Face model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_sections(sections):
    summaries = []
    for section in sections:
        summary = summarizer(section['content'][:1024], max_length=150, min_length=50, do_sample=False)[0]['summary_text']
        summaries.append(f"{section['law_type']} Chapter {section['chapter_number']} - {section['chapter_title']}: {summary}")
    return "\n".join(summaries)

# Function to provide crime reporting steps
def report_steps(query):
    steps = {
        "theft": [
            "1. Visit the nearest police station.",
            "2. File an FIR (First Information Report) detailing the theft.",
            "3. Provide any evidence such as CCTV footage or witness statements.",
            "4. Obtain a copy of the FIR for reference.",
            "5. Follow up with the police regarding the investigation."
        ],
        "assault": [
            "1. Seek medical attention if needed.",
            "2. Report the incident at the nearest police station.",
            "3. File an FIR with detailed information about the assault.",
            "4. Provide evidence such as medical reports or witness statements.",
            "5. Follow up with the authorities for updates on the case."
        ]
    }
    for crime, step_list in steps.items():
        if crime in query.lower():
            return "\n".join(step_list)
    return "Please visit the nearest police station for guidance."

# Load a lightweight LLM using pipeline
llm_pipeline = pipeline("text-generation", model="facebook/opt-1.3b")

def generate_response(query, law_type=None):
    sections = retrieve_sections(query, law_type)
    summary = summarize_sections(sections)
    steps = report_steps(query)

    prompt = f"""
    You are a legal expert providing guidance on Indian laws.
    A user has asked: "{query}"

    Based on relevant legal sections, provide a clear and structured response.

    Relevant Laws:
    {summary}

    Steps to Report:
    {steps}

    Explain the importance of the retrieved laws and how they apply to the user's case.
    """

    response = llm_pipeline(prompt, max_length=500, do_sample=True)[0]['generated_text']
    return response

# Example usage
load_faiss()
query = "What are the laws regarding theft?"
law_type = "IPC"  # User can specify "IPC" or "BNS"
print(generate_response(query, law_type))


Device set to use cuda:0
Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



    You are a legal expert providing guidance on Indian laws.
    A user has asked: "What are the laws regarding theft?"
    
    Based on relevant legal sections, provide a clear and structured response.
    
    Relevant Laws:
    IPC Chapter 17 - Offences Against Property: A person is said to cause a thing to move by removing an obstacle which prevented it from moving or by separating it from any other thing. The consent mentioned in the definition may be express or implied, and may be given either by the person in possession, or by the owner.
IPC Chapter 23 - Attempts to Commit Offences: Punishment for attempting to commit offences punishable with imprisonment for life or other imprisonment.Whoever attempts to commit an offence punishable by this Code with imprisonment or imprisonment, or to cause such an offence to be committed, shall be punished with imprisonment of any description.
    
    Steps to Report:
    1. Visit the nearest police station.
2. File an FIR (First Informat